In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os
import strgen

from tqdm import tqdm

from rnn.helpers import *
from rnn.model2 import *
from rnn.generate import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
file_path = 'language_data/Asimov.txt'
# file = unidecode.unidecode(open(file_path).read())
with open(file_path, encoding="utf8") as f:
    file = f.read()
file_len = len(file)
chunk_len = 400
batch_size = 200
n_epochs = 2000
hidden_size = 100
n_layers = 1
learning_rate = 0.001
model_type = 'lstm'
print_every = 50
plot_every = 50
filename = 'P2'

split = int(0.9*file_len)
train_text = file[:split]
test_text = file[split:]

def random_training_set(text, chunk_len, batch_size):
    input_data = torch.zeros(batch_size, chunk_len).long().to(device)
    target = torch.zeros(batch_size, chunk_len).long().to(device)
    for i in range(batch_size):
        start_index = random.randint(0, len(text) - chunk_len - 1)
        end_index = start_index + chunk_len + 1
        chunk = text[start_index:end_index]
        input_data[i] = char_tensor(chunk[:-1])
        target[i] = char_tensor(chunk[1:])
    return input_data, target

def train(inp, target):
    hidden = NET.init_hidden(batch_size)
    NET.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = NET(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    NET_optimizer.step()

    return loss.item() / chunk_len

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(NET, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

NET = RNN(
    n_characters,
    hidden_size,
    n_characters,
    model_type=model_type,
    n_layers=n_layers,
).to(device)
NET_optimizer = torch.optim.Adam(NET.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

print("Training for %d epochs..." % n_epochs)
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(test_text, chunk_len, batch_size))
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate(NET, 'Wh', 100, device=device), '\n')

save()



  0%|          | 0/2000 [00:00<?, ?it/s]

Training for 2000 epochs...


  1%|          | 14/2000 [00:25<1:01:15,  1.85s/it]


KeyboardInterrupt: 

In [9]:
print(len(file))

1278774
